In [ ]:
%load_ext autoreload
%autoreload 2
    
from manager import *

In [ ]:
manager = Manager()


In [ ]:
if (self.status != SCANNING and 
    self.status != ARM_RETURNED and 
    self.status != ERROR):
    manager.run_once()